# Data Collection & Preprocessing 

The code in this notebook is for the use of scaping the latest comment titles and body from a specific subreddit, converting to a dataframe, and exporting to a .csv file


### Import Libaries

In [1]:
import pandas as pd
import praw
import sensitive as sens
import db_sensitive as db_sens
from tqdm.notebook import tqdm
from sqlalchemy import create_engine, engine

### Reddit API Scrape

To use the below code and function you must create a reddit account [here](https://www.reddit.com/) and register for use of the API. The username and password will be from your general Reddit account while your client id, cleint secret, and user agent will be from the API token your create attached to the account.

In [2]:
# enter your personal account info in accordingly
reddit = praw.Reddit(
    client_id=sens.client_id,
    client_secret=sens.client_secret,
    password=sens.password, 
    user_agent=sens.user_agent,
    username=sens.username
)

The below function takes in the above-created praw instance, the subreddit you would like to scape as a string, and the number of latest posts you would like to collect as an integer. 

The output will print the number of comments successfully scaped and a dataframe with each row including the comments category, body, and title.

In [3]:
def reddit_scraper(praw_object, sub_reddit, num_posts):
    # create submissions object to iterate over
    submissions = praw_object.subreddit(sub_reddit).hot(limit = num_posts)
#     submission.comments.replace_more(limit=0) # flatten tree
# comments = submission.comments.list()
    
    
    # create list of dictionaries for easy conversion to df
    posts = []
    comments = []
    for i, post in enumerate(submissions):
        post.comments.replace_more(limit=0)
        comms = post.comments.list()
        posts.append({
            'category': post.subreddit.display_name,
            'title': post.title,
            'body': post.selftext,
            'sub': sub_reddit,
            'sub_post_id': i,
            'n_comments': len(comms)
        })
        for com in comms:
            comments.append({
                'sub_post_id': i,
                'text': com.body,
                'likes': com.likes,
            })
    
    # show number of articles collected and out df
    print(f'You collected {len(posts)} reddit comments about {sub_reddit}')
    return pd.DataFrame(posts), pd.DataFrame(comments)

In [4]:
subs = ['datascience', 'datasciencejobs', 'machinelearning', 
        'machinelearningjobs', 'learnmachinelearning',
        'learndatascience']
scraped = {}
for sub in tqdm(subs):
    scraped[sub] = reddit_scraper(reddit, sub, 100)

# # scape 1,000 articles on history and consipracy to model
# history_df = reddit_scapper(reddit, 'epicseven', 1_000)
# conspiracy_df = reddit_scapper(reddit, 'lewdseven', 1_000)

You collected 100 reddit comments about datascience
You collected 100 reddit comments about datasciencejobs
You collected 100 reddit comments about machinelearning
You collected 100 reddit comments about machinelearningjobs
You collected 100 reddit comments about learnmachinelearning
You collected 100 reddit comments about learndatascience



In [5]:
posts_l = []
comms_l = []
for _, tup in scraped.items():
    posts_l.append(tup[0])
    comms_l.append(tup[1])

In [6]:
posts_df = pd.concat(posts_l)
posts_df = posts_df.reset_index(drop=True)
comms_df = pd.concat(comms_l)
comms_df = comms_df.reset_index(drop=True)

In [7]:
# database connection
db_config = {
    'drivername':'postgresql',
    'host': db_sens.host,
    'password': db_sens.password,
    'username': db_sens.username,
    'port': db_sens.port,
    'database': db_sens.database,
}
db_engine = create_engine(engine.url.URL(**db_config))
posts_df.to_sql('posts', db_engine, if_exists='replace')
comms_df.to_sql('comms', db_engine, if_exists='replace')

# test loading back in

In [10]:
comms_read = pd.read_sql_table('comms', db_engine)
print(comms_read.shape)
comms_read.head()

(2589, 4)


,index,sub_post_id,text,likes
0,0,0,Learn SQL\n\nI’m always aghast at how many que...,None
1,1,0,"I'm a high school student(10th grade, 16) who'...",None
2,2,0,Tomorrow I am having a coffee with a Senior Da...,None
3,3,0,What salary range to give to a fortune 500 com...,None
4,4,0,Upcoming (this May) graduate with two BS degre...,None


In [11]:
posts_read = pd.read_sql_table('posts', db_engine)
print(posts_read.shape)
posts_read.head()

(600, 7)


,index,category,title,body,sub,sub_post_id,n_comments
0,0,datascience,Weekly Entering & Transitioning Thread | 08 Ma...,_Bleep Bloop_. Welcome to this week's entering...,datascience,0,86
1,1,datascience,Open COVID-19 Dataset,I was frustrated with the maintenance issues i...,datascience,1,13
2,2,datascience,Is there something like Project Euler but for ...,"Hello all, I've really enjoyed solving the pro...",datascience,2,17
3,3,datascience,Is there a website like leetcode but for data ...,Hi as you all know leetcode is a website for s...,datascience,3,3
4,4,datascience,Are there laws like the three laws of robotics...,I realise the topic is complex and can easily ...,datascience,4,1
